# 1. Загрузка исходных данных

Исходные данные в формате .txt. 

Перечень полей:

id|||состояние (действует/не действует)|||Номер заявки|||Категории МПК|||Дата подачи заявки|||Дата регистрации|||Автор|||Правообладатель|||Название|||Аннотация

In [ ]:
data_dir='.\\input_data\\'
file_name='!all_docs.txt'
vpw_name='all_vpw.txt'

# --- open single file !all_docs.txt then reorganize for vowpal wabbit format ---

with open(data_dir+file_name, "r") as fp:

    patents_info_lines = fp.readlines()
    
    with open(data_dir+vpw_name, 'w') as vpw_file:
        for patent in patents_info_lines:
            patent_list=patent.decode('utf8').split('|||')         
            
            mpks=patent_list[len(patent_list)-7].split('|')
            
            for mpk in mpks:
                
                mpk=mpk.strip()[:-10]  # delete (2006.01) and so on at the end of mpk
                mpk_list=[]
                mpk_list=mpk.split()
                mpks_main_vpw=mpks_main_vpw+mpk_list[0] + ' '
                mpks_vpw=mpks_vpw + mpk.replace(" ", "_") + ' '
                      
            authors=patent_list[len(patent_list)-4].split('|')
            authors_vpw=''
            for author in authors:
                author=author.replace("(RU)", "").rstrip()
                authors_vpw=authors_vpw + author.replace(" ", "_") + ' '
            # -
            
             # - assignees rewrite -
            assignees=patent_list[len(patent_list)-3].split('|')
            assignees_vpw=''
            for assignee in assignees:
                assignee=assignee.replace("(RU)", "").rstrip()
                assignees_vpw=assignees_vpw + assignee.replace(" ", "_") + ' '
            # -
            
            #  - title and abstract rewrite   - delete symbol "|"
            title=patent_list[len(patent_list)-2].replace("|", "")
            abstract=patent_list[len(patent_list)-1].replace("|", "")
            
            new_line=''
            new_line=patent_list[0]+' |type RUPM |' + 'status '+patent_list[1] + ' |mpk_main ' + mpks_main_vpw + '|mpk_full ' + mpks_vpw +\
            '|date_app ' + patent_list[len(patent_list)-6] + ' |date_pub ' + patent_list[len(patent_list)-5] + ' |authors ' + authors_vpw +\
            '|assignee ' + assignees_vpw + '|text ' + title + ' '+ abstract
            vpw_file.write(new_line.encode('utf8'))

Процедуры предобработки

In [ ]:
data_dir='.\\input_data\\'

import pymorphy2
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from collections import Counter

# -- стоп слова
stop_file="stop_extend.txt"
russian_stopwords = stopwords.words("russian")   
rsw = [unicode(word) for word in russian_stopwords]
rsw = rsw + open(data_dir+stop_file, "r").read().decode('cp1251').split()

#-- файлы с "белыми списками" - слова из названий программ импортозамещения: авиастроение, строительный_транспорт и т.п.
importTitles_unigrams_file="import_titles_unigrams.txt"
importTitles_bigrams_file="import_titles_bigrams.txt"
imp_uni=open(data_dir+importTitles_unigrams_file, "r").read().decode('cp1251').split()
imp_bi=open(data_dir+importTitles_bigrams_file, "r").read().decode('cp1251').split()

tokenizer = RegexpTokenizer(u'[A-Z|a-z|А-Я|Ё|а-я|ё|]+') # only words         
morph = pymorphy2.MorphAnalyzer()
    
def text_preprocess(plain_text):    
    intermediate = tokenizer.tokenize(plain_text.lower())
    intermediate = [i for i in intermediate if len(i)>2]
    intermediate = [morph.parse(i)[0].normal_form for i in intermediate]
    preprocessed_text = [i for i in intermediate if i not in rsw] 
    return preprocessed_text

def get_counted_unigrams(preprocessed_text):
    counts_uni = Counter(preprocessed_text)  # счетчик частот униграмм  
    unigrams_count_line='' # unigrams in line with counts
    import_unigrams_count_line=''  # отдельно выделяются слова из белого списка униграмм программы импортозамещения (из названия программ, например: аэродромный, автомобильный и т.п.)
    for char in counts_uni.keys():
        if char in imp_uni:    # если слово содержится в "белом" списке униграм программы импортозамещения (например: фармацевтический, авиастроение и т.п.)
            import_unigrams_count_line=import_unigrams_count_line+char+':'+str(counts_uni[char])+' '
        else:
            unigrams_count_line=unigrams_count_line+char+':'+str(counts_uni[char])+' '
    return import_unigrams_count_line, unigrams_count_line

def get_counted_bigrams(preprocessed_text): # Наиболее частотные биграммы с частотой большей 2. 
    # (также потом к "биграммам относились унигаммы с названием отрасли. например: транспортный, химический и т.д.")
    
    #- bigrams cut-off   --  сколько наиболее частотный биграмм брать, на какой частоте отсекать для  включения в словарь
    bigram_most_common_count=15
    bigramm_cutoff_value=2
    bigrams_count_line='' # bigrams in line with counts
    import_bigrams_count_line=''
    biword =  [b for b in nltk.bigrams(preprocessed_text)]
    counts_bi = Counter(biword)  # счетчик частот биграмм
    import_bigrams_count_line=''
    for char in counts_bi.keys():
        char_='_'.join(char)
        bigrams_count_line='' # bigrams in line with counts
        if char_ in imp_bi:
            import_bigrams_count_line=import_bigrams_count_line+char_+':'+str(counts_bi[char])+' '
        else:
            bigrams_common=counts_bi.most_common(bigram_most_common_count)
            
            for bigram in bigrams_common:
                if bigram[1]>=bigramm_cutoff_value:
                    bigrams_count_line=bigrams_count_line+ '_'.join(bigram[0]) +':' +str(bigram[1]) +' ' 
    return import_bigrams_count_line, bigrams_count_line

Подготовка файла в формате .vpw для основного файла данных

In [ ]:
data_dir='.\\input_data\\'
vpw_name='all_vpw2.txt'
vpw_lemmas_name='all_vpw_lemmas_bigrams2.txt'

bigram_most_common_count=15
bigramm_cutoff_value=3

import io
import pymorphy2
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from collections import Counter

# --- Main text preprocessing (counted unigrams, bigrams) ---
with open(data_dir+vpw_name, "r") as f:
    
    patents_info_lines = f.readlines()
        
    with io.open(data_dir+vpw_lemmas_name, 'w', encoding='utf8') as vpw_lemmas_file:  #file to write lemmas and bigramms
        for patent in patents_info_lines:
            patent_list=patent.decode('utf8').split('|')
            text = patent_list[len(patent_list)-1][5:]  # patent text (title+abstract) is the last one;  [5:] - вырезаем из начала "text "
            preprocessed_text=text_preprocess(text) 
            import_unigrams_count_line, unigrams_count_line=get_counted_unigrams(preprocessed_text)
            import_bigrams_count_line, bigrams_count_line=get_counted_bigrams(preprocessed_text)
        
            # modalities not changed:
            old_line='|'.join(patent_list[:-1])
            new_line=old_line+' |text ' + unigrams_count_line + ' |bigrams ' + import_unigrams_count_line + import_bigrams_count_line + bigrams_count_line
            vpw_lemmas_file.write(new_line)
            vpw_lemmas_file.write(unicode('\n'))
      

Для 22 файлов .txt по отраслям импортозамещения - подготовить для каждого файл в формате .vpw

In [ ]:
# -----   From Import program prepare 22 different vowpal wabbit files--- 
# ----- !!! Prepare 22 different files !! ----
import io
import glob
import codecs

data_dir='.\\import_vpw_and_batches\\'

files = glob.glob("import_text_files/*.txt")
for imp_file in files:
    file_name=imp_file.split("\\")[1].split(".")[0].replace(' ', '_')  # get file name like "aerodromn_tehn"
    file_name=file_name+".txt"
    with codecs.open(imp_file, 'r', 'cp1251') as f:
        import_file_in_line = " ".join(line.strip().replace('|', ' ') for line in f)
        preprocessed_text=text_preprocess(import_file_in_line)
        import_unigrams_count_line, unigrams_count_line=get_counted_unigrams(preprocessed_text)
        import_bigrams_count_line, bigrams_count_line=get_counted_bigrams(preprocessed_text)
        with io.open(data_dir+file_name, 'w', encoding='utf8') as f_out:
            f_out.write('|text ' + unigrams_count_line + ' |bigrams ' + import_unigrams_count_line + import_bigrams_count_line + bigrams_count_line + '\n')

Для 22 файлов .vpw (по отраслям импортозамещения) создать 22 словаря, по которым потом будем потом вводить регуляризаторы сглаживания

In [ ]:
#-- Build 22 Bigartm batches and 22 dictionary for IMPORT files
import artm
import glob

data_dir='.\\import_vpw_and_batches\\'

files = glob.glob("import_vpw_and_batches/*.txt")
for imp_file in files:
    file_name_txt=imp_file.split("\\")[1]
    file_name=file_name_txt.split('.')[0]
    target_folder=file_name+'_batches'
    batch_vectorizer_import = artm.BatchVectorizer(data_path=data_dir+file_name_txt,
                                        data_format='vowpal_wabbit',
                                        target_folder=data_dir+target_folder,
                                        batch_size=100)
    import_dictionary = artm.Dictionary()
    dictionary_name=file_name+'_dictionary'
    dictionary_name_txt=dictionary_name+'.txt'
    import_dictionary.gather(data_path=data_dir+target_folder)
    import_dictionary.save(dictionary_path=data_dir+dictionary_name)
    import_dictionary.save_text(dictionary_path=data_dir+dictionary_name_txt)

In [ ]:
# --- create batches and MAIN dictionary 
# !!!!!!!!!!!!!!!!!!!!!!!!!!! We need it only once !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
import artm

data_dir='.\\input_data\\'
data_file='all_vpw_lemmas_bigrams.txt'

# ----  BIGARTM !!! ------
batch_vectorizer = artm.BatchVectorizer(data_path=data_dir+data_file,
                                        data_format='vowpal_wabbit',
                                        target_folder='collection_batches',
                                        class_ids=['text', 'bigrams'] #, 'mpk_main', 'mpk_full']
                                       )

main_dictionary = artm.Dictionary()
main_dictionary.gather(data_path='collection_batches')
main_dictionary.save(dictionary_path='collection_batches/main_dictionary')
main_dictionary.save_text(dictionary_path='collection_batches/main_dictionary.txt')


In [ ]:
#  -- Filter MAIN dictionary from not specifield words  --
#сохранить токены с df=[1,3500)

main_dictionary.load_text(dictionary_path='collection_batches/main_dictionary.txt')
main_dictionary.filter(class_id='text', min_df=1, max_df=3500)   #сохранить токены с df=[1,3500) !! предварительно сохранив копию, т.к. фильтр вносит изменения сразу в файл
main_dictionary.save_text(dictionary_path='collection_batches/main_dictionary.txt')

# просматриваем получившийся файл в панде
data_dir='.\\collection_batches\\'
data_file='main_dictionary_for_pandas.txt'  # отличается от main_dictionary.txt удалением первых двух служебных строк
header=['token', 'class_id', 'value', 'tf', 'df']
main_pd = pd.read_csv(data_dir+data_file, delimiter=',', names=header, encoding="utf8")
main_pd = main_pd.sort_values(by ='df', ascending=False)
main_pd.head(200)

In [ ]:
# ------  Edit Import Dictionary  --  Set value= 2.0 for all import-words
# ------- Edit Main Dictionary    --  Set value -1.0 for all words

data_dir='.\\collection_batches\\'
import_dir='.\\import_batches\\'

data_file='main_dictionary_for_pandas.txt'
import_file='import_dictionary_filtered_from_pandas.txt'

header=['token', 'class_id', 'value', 'tf', 'df']

main_pd = pd.read_csv(data_dir+data_file, delimiter=',', names=header, encoding="utf8")
import_pd = pd.read_csv(import_dir+import_file, delimiter=',', names=header, encoding="utf8")
import_list = import_pd["token"].tolist()[2:]  # убираем первые 2 строчки
main_with_import_pd=main_pd[main_pd['token'].isin(import_list)]  # строки main_dict у которых token из import словаря
main_with_import_pd.loc[:,'value']=2
main_others_pd=main_pd[~main_pd['token'].isin(import_list)]  # токены не из импорт словаря
main_others_pd.loc[:,'value']=-1

main_edited_final_pd = pd.concat([main_with_import_pd, main_others_pd], ignore_index=True, sort =False)

out_file= 'edited_dictionary_2_-1.txt'
main_edited_final_pd.to_csv(data_dir+out_file, sep=',', encoding='utf8', index=False, header=False)

# !!!  в финальном файле руками добавить пробелы перед запятыми и первые две строчки

In [ ]:
main_dictionary.save(dictionary_path='collection_batches/main_dictionary')